In [2]:
import numpy as np
from tqdm import tqdm
from scipy.stats import multivariate_normal, matrix_normal, norm

# Data generation

In [3]:
# setting
ControlList = ['InControl','OutControl']
DataTypeList = ['Profile','Image']
CovList = ['TriDiagonal','Exponential']
DistList = ['Normal','Exponential']
phiList = [0.3, 0.7]
rhoList = [0.3]

In [4]:
def getCov(rho, p, type):
    Sigma = np.eye(p)
    if type == CovList[0]:
        tmp = np.ones(p-1)*rho
        Sigma = Sigma+np.diag(tmp, k = 1)+np.diag(tmp, k = -1)
    elif type == CovList[1]:
        for d in range(1, p-1):
            tmp = np.ones(p-d)*(rho**d)
            Sigma = Sigma+np.diag(tmp, k = d)+np.diag(tmp, k = -d)
    return Sigma

## Type 1

### In-control

In [52]:
seedid = 19940402
np.random.seed(seedid)
p = 200
w = 5
TrnLen = 1000 # full scale is 50000
ARL0= 1000
mu0 = 5*np.ones(p)
ControlType = 'InControl'
DataType = 'Profile'
DistType = 'Normal'
rho = 0.3 
CovType = 'TriDiagonal'
phi = 0.3
Tag = (ControlType,DataType,CovType,DistType,'phi',str(phi),'rho',str(rho))
TagJoin = ''.join(Tag)
print(Tag)
trn = []
Cov = getCov(rho,p,CovType)
Phi = phi * np.eye(p)   

eps0 = multivariate_normal.rvs(np.zeros(p), Cov)
for i in tqdm(range(TrnLen)):
    eps = np.matmul(Phi, eps0) + multivariate_normal.rvs(np.zeros(p), Cov)
    trn.append(mu0 + eps)
    eps0 = eps
trn = np.array(trn)
np.save("data/in-control/Trn"+TagJoin+".npy", trn)

('InControl', 'Profile', 'TriDiagonal', 'Normal', 'phi', '0.3', 'rho', '0.3')


100%|██████████| 1000/1000 [00:30<00:00, 32.34it/s]


### Out-of-control

In [13]:
p = 200
w = 5
deltanorm = 2 # the norm of shift for Type1
ShiftType = 'Sparse'

if ShiftType == 'Sparse':
    shift_basis = np.zeros(p) # sparse
    shift_basis[18:23] = np.ones(5)
    shift_basis = deltanorm/np.sqrt(np.sum(shift_basis**2))*shift_basis

elif ShiftType == 'Stepwise':
    shift_basis = np.zeros(p) # piece_1
    shift_basis[50:100] = np.ones(50)
    shift_basis[100:150] = 2 * np.ones(50)
    shift_basis[150:200] = 3 * np.ones(50)
    shift_basis = deltanorm/np.sqrt(np.sum(shift_basis**2))*shift_basis

elif ShiftType == 'Zigzag':
    shift_basis = np.zeros(p) # zigzag
    left = 1
    right = -1
    for j in range(10):
        shift_basis[j*20:(j+1)*20] = np.linspace(left, right, 20)
        left, right = right, left
    shift_basis = deltanorm/np.sqrt(np.sum(shift_basis**2))*shift_basis     


In [30]:
seedid = 19940402
# np.random.seed(seedid)
TrnLen = 1000 # length of sequence
SeqNum = 10 # number of sequence, full scale is 1000
ARL0 = 1000
mu0 = 5*np.ones(p)
mu1 = mu0+shift_basis
ControlType = 'OutControl'
DataType = 'Profile'
DistType = 'Normal'
rho = 0.3 
CovType = 'TriDiagonal'
phi = 0.3
Tag = (ControlType,DataType,CovType,DistType,ShiftType,'phi',str(phi),'rho',str(rho),'deltanorm',str(deltanorm))
TagJoin = ''.join(Tag)
print(Tag)

Cov = getCov(rho,p,CovType)
Phi = phi * np.eye(p)   


trn = []
for n in tqdm(range(SeqNum)):
    trn_seq = []
    eps0 = multivariate_normal.rvs(np.zeros(p), Cov)
    for i in range(TrnLen):
        eps = np.matmul(Phi, eps0) + multivariate_normal.rvs(np.zeros(p), Cov)
        trn_seq.append(mu1 + eps)
        eps0 = eps
    trn.append(trn_seq)
trn = np.array(trn)

np.save("data/out-of-control/Trn"+TagJoin+".npy", trn)

('OutControl', 'Profile', 'TriDiagonal', 'Normal', 'Sparse', 'phi', '0.3', 'rho', '0.3', 'deltanorm', '2')


100%|██████████| 10/10 [04:52<00:00, 29.30s/it]


In [31]:
trn.shape #(numseq,lenseq,dimofprofile)

(10, 1000, 200)

## Type 2

### In-control

In [70]:
seedid = 19940402
np.random.seed(seedid)
p = 200
w = 100
TrnLen = 4000 # full scale is 30000
ARL0 = 1000

ControlType = 'InControl'
DataType = 'Image'
DistType = 'Normal'
phi = 0.3
rho = 0.3 
Phi = phi * np.eye(p)
m = 5*np.ones([w,p])
CovType = 'TriDiagonal'

TagN = (ControlType,DataType,CovType,'Normal','phi',str(phi),'rho',str(rho))
TagNJoin = ''.join(TagN)
TagE = (ControlType,DataType,CovType,'Exponential','phi',str(phi),'rho',str(rho))
TagEJoin = ''.join(TagE)
print(CovType)
Covw = getCov(rho,w,CovType)
Covp = getCov(rho,p,CovType)

trnN = []
trnE = []
for i in tqdm(range(TrnLen)):
    eps = matrix_normal.rvs(mean=np.zeros([w,p]), rowcov=Covw, colcov=Covp)
    trnN.append(m + eps)
    trnE.append(m + (-1 * np.log(1 - norm.cdf(eps))))
trnN = np.array(trnN)
trnE = np.array(trnE)

np.save("data/in-control/Trn"+TagNJoin+".npy", trnN)
np.save("data/in-control/Trn"+TagEJoin+".npy", trnE)

TriDiagonal


  0%|          | 0/4000 [00:00<?, ?it/s]

100%|██████████| 4000/4000 [00:14<00:00, 281.99it/s]


### Out-of-control

In [11]:
p = 200
w = 100
DeltaNorm = 10 # the norm of shift for Type2
ShiftType = 'Chessboard'

if ShiftType == 'Chessboard':
    # Chessboard
    shift_matrix = []
    element = [[0, 1, 0, -1], [-1, 0, 1, 0]]
    for i in range(w // 5):
        row_element = element[i % 2]
        row = []
        for j in range(p // 10):
            row += [row_element[j % 4]] * 10
        shift_matrix += [row] * 5
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Sparse':
    # Sparse
    shift_matrix = np.zeros([w, p])
    shift_matrix[8:13, 18:23] = np.ones([5, 5])

    shift_matrix = np.zeros([w, p])
    for i in range(99):
        shift_matrix[i:(i+2), (2*i):(2*i+2)] = np.ones([2,2])

    shift_matrix = np.zeros([w, p])
    for i in range(25, 80):
        j = int((i**2) * 100 / 80**2)
        shift_matrix[i:(i+3), j:(j+3)] = np.ones([3,3])

    u1 = np.linspace(1, -1, w)
    v1 = np.linspace(1, 0, p)
    shift_matrix = np.matmul(u1.reshape([w, 1]), v1.reshape([1, p]))
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Sine(row)':
    # Sine (row)
    shift_matrix = np.zeros([w, p])
    for i in range(30):
        shift_matrix[i,:] = np.sin(np.arange(0, p) * np.pi * 2 / 20)
    for i in range(30, 60):
        shift_matrix[i,:] = np.sin(np.arange(0, p) * np.pi * 4 / 20)
    for i in range(60, 100):
        shift_matrix[i,:] = np.sin(np.arange(0, p) * np.pi * 6 / 20)
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Sine(col)':
    # Sine (column)
    shift_matrix = np.zeros([w, p])
    for i in range(60):
        shift_matrix[:,i] = np.sin(np.arange(0, w) * np.pi * 2 / 10)
    for i in range(60, 120):
        shift_matrix[:,i] = np.sin(np.arange(0, w) * np.pi * 4 / 10)
    for i in range(120, 200):
        shift_matrix[:,i] = np.sin(np.arange(0, w) * np.pi * 6 / 10)
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Ring':
    # Ring
    shift_matrix = np.zeros([w, p])
    for i in range(w):
        for j in range(p):
            if int(np.sqrt((i-50)**2 + (j-100)**2)) % 12 <= 3:
                shift_matrix[i, j] = 1
            elif int(np.sqrt((i-50)**2 + (j-100)**2)) % 12 >= 8:
                shift_matrix[i, j] = -1
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)

In [12]:
seedid = 19940402
np.random.seed(seedid)
TrnLen = 10 #30000
SeqLen = 5 # full scale is 1000
ARL0 = 1000

ControlType = 'OutControl'
DataType = 'Image'
DistType = 'Normal'
phi = 0.3
rho = 0.3 
Phi = phi * np.eye(p)
m = 5*np.ones([w,p])
CovType = 'TriDiagonal'

TagN = (ControlType,DataType,CovType,ShiftType,'Normal','phi',str(phi),'rho',str(rho),'DeltaNorm',str(DeltaNorm))
TagNJoin = ''.join(TagN)
TagE = (ControlType,DataType,CovType,ShiftType,'Exponential','phi',str(phi),'rho',str(rho),'DeltaNorm',str(DeltaNorm))
TagEJoin = ''.join(TagE)
print(CovType)
Covw = getCov(rho,w,CovType)
Covp = getCov(rho,p,CovType)

trnN = []
trnE = []
for n in tqdm(range(SeqLen)):
    trnN_seq = []
    trnE_seq = []
    for i in range(TrnLen):
        eps = matrix_normal.rvs(mean=np.zeros([w,p]), rowcov=Covw, colcov=Covp)
        trnN_seq.append(m + eps)
        trnE_seq.append(m + (-1 * np.log(1 - norm.cdf(eps))))
    trnN.append(trnN_seq)
    trnE.append(trnE_seq)

trnN = np.array(trnN)
trnE = np.array(trnE)

np.save("data/out-of-control/Trn"+TagNJoin+".npy", trnN)
np.save("data/out-of-control/Trn"+TagEJoin+".npy", trnE)    

TriDiagonal


100%|██████████| 5/5 [00:00<00:00, 26.73it/s]


In [13]:
trnN.shape

(5, 10, 100, 200)

# MGLR

## Functions

In [32]:
class MGLR:
    def __init__(self, target_arl0 = 1000, region_row=5, region_col=20, m=10, window_size=5, skip_row=0):
        self.target_arl0 = target_arl0
        self.region_row = region_row
        self.region_col = region_col
        self.m = m
        self.window_size = window_size
        self.skip_row = skip_row
    
    def getX(self, Y):
        X = []
        for i in range(len(Y) // self.region_row):
            for j in range(len(Y[0]) // self.region_col):
                X.append(np.mean(Y[(i*self.region_row):((i+1)*self.region_row), (j*self.region_col):((j+1)*self.region_col)]))
        return X
        
    
    def setup(self, data_in):
        col_num = data_in.shape[1]
#         self.M0 = np.zeros([self.window_size, col_num])
#         for i in np.arange(0, data_in.shape[0] - self.window_size + 1, self.skip_row + 1):
#             self.M0 += data_in[i:(i+self.window_size)]
#         self.M0 /= (data_in.shape[0] - self.window_size + 1) // (self.skip_row + 1)
        self.M0 = 5 * np.ones([self.window_size, col_num])
        
        X = []
        for i in np.arange(0, data_in.shape[0] - self.window_size + 1, self.skip_row + 1):
            Y = data_in[i:(i+self.window_size)]
            X.append(self.getX(Y-self.M0))
        self.mu0 = np.mean(X, axis=0)
        self.inv_sigma0 = np.linalg.inv(np.cov(X, rowvar=False))
        self.control_limit = 79.39
        
        
    def monitor(self, data, Y_0, max_RL=10000):
        col_num = data.shape[1]
        X = []
        mlr = 0
        k = 0
        i = 0
        self.RL = 0
        while mlr <= self.control_limit and self.RL <= max_RL:
            if i + self.window_size > len(data):
                Y = Y[(self.skip_row+1):]
                while len(Y) < self.window_size:
                    Y = np.append(Y, [Phi[0,0] * Y[-1] + (1 - Phi[0,0]) * Y_0 
                                      + multivariate_normal.rvs(np.zeros(col_num), cov_error)], axis=0)
            else:
                Y = data[i:(i+self.window_size)]
            k += 1
            X.append(self.getX(Y-self.M0))
            lr = []
            for tau in range(max([0, k-self.m]), k):
                mu1 = np.sum(X[(tau):], 0) / (k-tau)
                diff = mu1 - self.mu0
                lr.append((k-tau)/2 * np.matmul(np.matmul(diff.T, self.inv_sigma0), diff))
            mlr = np.max(lr)
            self.RL += 1
            i += self.skip_row + 1
    
    def monitor_image(self, M, max_RL=10000):
        X = []
        mlr = 0
        k = 0
        self.RL = 0
        while mlr <= self.control_limit and self.RL <= max_RL:
            k += 1
            Y = matrix_normal.rvs(mean=M, rowcov=cov_row, colcov=cov_col, size=1)
            X.append(self.getX(Y-self.M0))
            lr = []
            for tau in range(max([0, k-self.m]), k):
                mu1 = np.sum(X[(tau):], 0) / (k-tau)
                diff = mu1 - self.mu0
                lr.append((k-tau)/2 * np.matmul(np.matmul(diff.T, self.inv_sigma0), diff))
            mlr = np.max(lr)
            self.RL += 1
    
    def monitor_exp_image(self, M, max_RL=10000):
        X = []
        mlr = 0
        k = 0
        self.RL = 0
        while mlr <= self.control_limit and self.RL <= max_RL:
            k += 1
            norm_epsilon = matrix_normal.rvs(mean=np.zeros([w, p]), rowcov=cov_row, colcov=cov_col)
            Y = M + (-1 * np.log(1 - norm.cdf(norm_epsilon)))
            X.append(self.getX(Y-self.M0))
            lr = []
            for tau in range(max([0, k-self.m]), k):
                mu1 = np.sum(X[(tau):], 0) / (k-tau)
                diff = mu1 - self.mu0
                lr.append((k-tau)/2 * np.matmul(np.matmul(diff.T, self.inv_sigma0), diff))
            mlr = np.max(lr)
            self.RL += 1

## Type 1

In [53]:
seedid = 19940402
np.random.seed(seedid)
p = 200
w = 5
TrnLen = 1000 #50000
ARL0=1000
mu0 = 5*np.ones(p)
ControlType = 'InControl'
DataType = 'Profile'
DistType = 'Normal'
rho = 0.3 
CovType = 'Tridiagonal'
phi = 0.3
Tag = (ControlType,DataType,CovType,DistType,'phi',str(phi),'rho',str(rho))
TagJoin = ''.join(Tag)
data_in = np.load("data/in-control/Trn"+TagJoin+".npy")
data_in.shape

(1000, 200)

In [54]:
ControlType = 'OutControl'
DataType = 'Profile'
DistType = 'Normal'
rho = 0.3 
CovType = 'TriDiagonal'
phi = 0.3
deltanorm = 2 # the norm of shift for Type1
ShiftType = 'Sparse'

Tag = (ControlType,DataType,CovType,DistType,ShiftType,'phi',str(phi),'rho',str(rho),'deltanorm',str(deltanorm))
TagJoin = ''.join(Tag)
print(Tag)
data_out = np.load("data/out-of-control/Trn"+TagJoin+".npy")
data_out.shape

('OutControl', 'Profile', 'TriDiagonal', 'Normal', 'Sparse', 'phi', '0.3', 'rho', '0.3', 'deltanorm', '2')


(10, 1000, 200)

In [55]:
mglr = MGLR()
mglr.setup(data_in)
mglr.control_limit = 62.63 # refer to Table 1 in paper

In [56]:
RL = []
cov_error = getCov(rho, p, CovType)
for i in tqdm(range(len(data_out))):
    mglr.monitor(data_out[i],5*np.ones(p),max_RL=10000)
    RL.append(mglr.RL)

100%|██████████| 10/10 [00:41<00:00,  4.19s/it]


In [57]:
[np.mean(RL), np.std(RL)]

[334.5, 690.1300239809887]

## Type 2

In [71]:
ControlType = 'InControl'
DataType = 'Image'
DistType = 'Normal'
phi = 0.3
rho = 0.3 
Phi = phi * np.eye(p)
m = 5*np.ones([w,p])
CovType = 'TriDiagonal'
DeltaNorm = 10 # the norm of shift for Type2
ShiftType = 'Chessboard'

TagN = (ControlType,DataType,CovType,'Normal','phi',str(phi),'rho',str(rho))
TagNJoin = ''.join(TagN)

data_in = np.load("data/in-control/Trn"+TagNJoin+".npy")
TrnLen,w,p = data_in.shape
TrnLen,w,p

(4000, 100, 200)

In [72]:
data_in = data_in.reshape(TrnLen*w,p)
data_in.shape

(400000, 200)

In [73]:
DeltaNorm = 10 # the norm of shift for Type2
ShiftType = 'Chessboard'

if ShiftType == 'Chessboard':
    # Chessboard
    shift_matrix = []
    element = [[0, 1, 0, -1], [-1, 0, 1, 0]]
    for i in range(w // 5):
        row_element = element[i % 2]
        row = []
        for j in range(p // 10):
            row += [row_element[j % 4]] * 10
        shift_matrix += [row] * 5
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Sparse':
    # Sparse
    shift_matrix = np.zeros([w, p])
    shift_matrix[8:13, 18:23] = np.ones([5, 5])

    shift_matrix = np.zeros([w, p])
    for i in range(99):
        shift_matrix[i:(i+2), (2*i):(2*i+2)] = np.ones([2,2])

    shift_matrix = np.zeros([w, p])
    for i in range(25, 80):
        j = int((i**2) * 100 / 80**2)
        shift_matrix[i:(i+3), j:(j+3)] = np.ones([3,3])

    u1 = np.linspace(1, -1, w)
    v1 = np.linspace(1, 0, p)
    shift_matrix = np.matmul(u1.reshape([w, 1]), v1.reshape([1, p]))
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Sine(row)':
    # Sine (row)
    shift_matrix = np.zeros([w, p])
    for i in range(30):
        shift_matrix[i,:] = np.sin(np.arange(0, p) * np.pi * 2 / 20)
    for i in range(30, 60):
        shift_matrix[i,:] = np.sin(np.arange(0, p) * np.pi * 4 / 20)
    for i in range(60, 100):
        shift_matrix[i,:] = np.sin(np.arange(0, p) * np.pi * 6 / 20)
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Sine(col)':
    # Sine (column)
    shift_matrix = np.zeros([w, p])
    for i in range(60):
        shift_matrix[:,i] = np.sin(np.arange(0, w) * np.pi * 2 / 10)
    for i in range(60, 120):
        shift_matrix[:,i] = np.sin(np.arange(0, w) * np.pi * 4 / 10)
    for i in range(120, 200):
        shift_matrix[:,i] = np.sin(np.arange(0, w) * np.pi * 6 / 10)
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Ring':
    # Ring
    shift_matrix = np.zeros([w, p])
    for i in range(w):
        for j in range(p):
            if int(np.sqrt((i-50)**2 + (j-100)**2)) % 12 <= 3:
                shift_matrix[i, j] = 1
            elif int(np.sqrt((i-50)**2 + (j-100)**2)) % 12 >= 8:
                shift_matrix[i, j] = -1
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)

M_shifted = 5*np.ones([w,p])+shift_matrix

In [74]:
mglr = MGLR(region_row=10, window_size=w, skip_row=w-1)
mglr.setup(data_in)

In [75]:
mglr.control_limit = 79.39

In [77]:
cov_col = getCov(phi, p, CovType)
cov_row = getCov(phi, w, CovType)
RL = []
for i in tqdm(range(20)):
#     np.random.seed(20170225 + i)  # comment when testing ARL1
    mglr.monitor_image(5*np.ones([w,p]), max_RL=10000)
    RL.append(mglr.RL)
#     if len(RL) % 100 == 0:
#         print(np.mean(RL))

100%|██████████| 20/20 [00:25<00:00,  1.28s/it]


In [78]:
[np.mean(RL), np.std(RL)]

[341.3, 204.968802504186]